# Data import

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
listings = pd.read_csv('../data/data_jan_listings_3plus.csv',low_memory=False)

In [3]:
def str_to_price(strn):
    '''The function converts a price entry from string to float and removes the $ character'''
    
    if type(strn)== str:
        return float(strn.strip('$').replace(',',''))
    return strn


def price_tonumerical(df, price_columns):
    '''This function takes as input a dataframe and a list of price column names and converts the columnns to
    floats'''
    
    for column in price_columns:
        df[[column]] = df[[column]].applymap(str_to_price)
    return df

def calculate_cleaning_ratio(df):
    '''This function calculates the ratio of cleaning fee to the nightly price'''
    
    df['cleaning_ratio']=round(df['cleaning_fee']/df['price'],2)
    return df

def calculate_deposit_ratio(df):
    '''This function calculates the ratio of security deposit fee to the nightly price'''
    
    df['deposit_ratio']=round(df['security_deposit']/df['price'],2)
    return df


price_columns=['price', 'security_deposit', 'cleaning_fee', 'extra_people']
price_tonumerical(listings, price_columns)
calculate_cleaning_ratio(listings)


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,host_since,host_location,host_about,host_response_time,host_response_rate,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,street,neighbourhood_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month,cleaning_ratio
0,11551.0,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,No Smoking (very strict) Check-in time is afte...,Tons of buses (24hrs) go into central London f...,Guest will have access to the entire apartment,No interaction with guests as you book the ent...,No Smoking (very strict) No pets are allowed i...,2009-10-03,"London, England, United Kingdom","Hello, I'm a friendly Italian man with a very ...",within an hour,100%,Brixton,3.0,3.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,"London, United Kingdom",Lambeth,SW9 8DG,51.46225,-0.11732000000000001,t,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Paid par...",88.0,$645.00,"$2,350.00",400.0,30.0,2.0,25.0,2.0,180.0,17.0,42,72,344,186,3.0,2010-03-21,2019-11-17,92.0,9.0,9.0,10.0,10.0,9.0,9.0,t,strict_14_with_grace_period,f,t,1.56,0.34
1,13913.0,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,For art lovers I can give guest my Tate Member...,The flat only a 10 minute walk to Finsbury Par...,Guest will have access to the self catering ki...,I like to have little chats with my guest over...,I'm an artist and have my artwork up on the wa...,2009-11-16,"London, England, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a few hours,70%,LB of Islington,4.0,4.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,"Islington, Greater London, United Kingdom",Islington,N4 3,51.568020000000004,-0.11120999999999999,t,Apartment,Private room,2.0,1.0,1.0,0.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",65.0,$333.00,"$1,176.00",100.0,15.0,1.0,15.0,1.0,29.0,30.0,60,90,365,20,6.0,2010-08-18,2019-11-25,97.0,10.0,10.0,10.0,10.0,9.0,9.0,f,moderate,f,f,0.17,0.23
2,15400.0,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Bright Chelsea Apartment This is a bright one...,Lots of windows and light. St Luke's Gardens ...,romantic,It is Chelsea.,The building next door is in the process of be...,The underground stations are South Kensington ...,There are two wardrobes for guests exclusive u...,If I am in the country I like to welcome my gu...,NO SMOKING PLEASE.. No unauthorised guests. No...,2009-12-05,"Kensington, England, United Kingdom","English, grandmother, I have travelled quite ...",within a few hours,100%,Chelsea,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,"London, United Kingdom",Kensington and Chelsea,SW3,51.48796,-0.16898,t,Apartment,Entire home/apt,2.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,$600.00,"$2,250.00",150.0,50.0,2.0,0.

# Clusters database building

In [54]:
def price_cat(x,pctl):
    '''Transforms an absolute price into a price category, given a list of percentiles'''
    if x <pctl[1]:
        return 'very_cheap'
    elif x >=pctl[1] and x <pctl[3]:
        return 'cheap'
    elif x >=pctl[3] and x <pctl[5]:
        return 'average'
    elif x >=pctl[5] and x <pctl[7]:
        return 'expensive'
    return 'very_expensive'

def clustering(data):
    '''Returns a dataframe containing the relevant clustering variables'''
    #pctl=[]
    columns=['location', 'price-boroughwise', 'property_type', 'review_scores_rating']
    cluster_df = pd.DataFrame(columns=columns)
    #for p in range(1,10):
    #    pctl.append(np.nanpercentile(data['price'], 10*p))
    #
    #cluster_df['price-citywise']=data['price'].apply(lambda x:price_cat(x,pctl))
    percentiles=dict()
    neighbourhoods = listings['neighbourhood_cleansed'].unique()
    for neigh in neighbourhoods:
        neigh_listings = listings[listings['neighbourhood_cleansed'] == neigh]
        indices= neigh_listings.index.values.tolist()
        pctl=[]
        for p in range(1,10):
            pctl.append(np.nanpercentile(neigh_listings['price'], 10*p))
        #neigh_listings['price'].apply(lambda x:price_cat(x,pctl))
        percentiles[neigh] = pctl
        for index in indices:
            cluster_df.loc[index,'price-boroughwise']=price_cat(neigh_listings.loc[index,'price'],pctl)
            
    
    bedroom_convert = {0.0: 'small',
                  1.0: 'small',
                  2.0: 'large',
                  3.0: 'large', 4.0: 'large',5.0: 'large',6.0: 'large',7.0: 'large',
                   8.0: 'large',9.0: 'large',10.0: 'large',11.0: 'large',
                   16.0: 'large',22.0: 'large'}
    cluster_df['property_type'] =  data['bedrooms'].map(bedroom_convert)
    cluster_df['room/apt']=data[['room_type']].applymap(lambda x: 'apartment' if x == 'Entire home/apt' else 'room')# if x == 'Private room' else 'Other'))
    
    cluster_df['location']=data['neighbourhood_cleansed']
    
    cluster_df['review_scores_rating']=data['review_scores_rating']
    
    cluster_df['property_type'] = cluster_df.apply(lambda x: 'room' if x['room/apt']== 'room' else x['property_type'], axis=1)
    cluster_df['listing_id'] = data['id']
    cluster_df['lat'] = data['latitude']
    cluster_df['lon'] = data['longitude']
    cluster_df=cluster_df.drop(columns='room/apt')
    cluster_df['cluster'] = "L:" + cluster_df['location'] + "_P:" + cluster_df['price-boroughwise'] + "_S:" + cluster_df['property_type']
    #matching_table = cluster_df.copy()
    #matching_table = matching_table.groupby(['location','price-boroughwise','property_type']).count().reset_index().drop(columns='review_scores_rating')
    #matching_table['cluster_id'] = matching_table.index
    
    return cluster_df, percentiles

clusters, percentiles=clustering(listings)


/Users/lcavicchia/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1368: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [55]:
clusters

,location,price-boroughwise,property_type,review_scores_rating,listing_id,lat,lon,cluster
0,Lambeth,expensive,small,92.0,11551.0,51.46225,-0.11732000000000001,L:Lambeth_P:expensive_S:small
33,Lambeth,cheap,room,98.0,53242.0,51.47075,-0.12913,L:Lambeth_P:cheap_S:room
34,Lambeth,expensive,small,96.0,53245.0,51.47246,-0.12727,L:Lambeth_P:expensive_S:small
50,Lambeth,cheap,room,96.0,72913.0,51.48064,-0.11439,L:Lambeth_P:cheap_S:room
51,Lambeth,cheap,large,98.0,73431.0,51.46295,-0.11199999999999999,L:Lambeth_P:cheap_S:large
...,...,...,...,...,...,...,...,...
51047,Sutton,expensive,small,100.0,39963193.0,51.3577,-0.13601,L:Sutton_P:expensive_S:small
51074,Sutton,expensive,small,98.0,39988769.0,51.35737,-0.21677,L:Sutton_P:expensive_S:small
51098,Sutton,very_expensive,small,40.0,40019636.0,51.37853,-0.21886999999999998,L:Sutton_P:very_expensive_S:small
51255,Sutton,expensive,large,60.0,40147104.0,51.34812,-0.19767,L:Sutton_P:expensive_S:large


In [6]:
def listing_to_cluster(listing_id, clusters):
    cluster_id = clusters[clusters['listing_id']==listing_id].iloc[0]['cluster']
    
    return cluster_id
    

In [7]:
location = clusters[clusters['listing_id']==53242][['location']]
price = clusters[clusters['listing_id']==53242][['price-boroughwise']]
ptype = clusters[clusters['listing_id']==53242][['property_type']]
cluster = matching_table[(matching_table['location'] == location ) &  (matching_table['price-boroughwise'] == price ) &\
              (matching_table['property_type'] == ptype )\
            ]
cluster

NameError: name 'matching_table' is not defined

In [ ]:
listing_to_cluster(53242, clusters)

In [56]:
clusters.to_csv('../data/jan/clusters.csv')
#matching_table.to_csv('../data/jan/clusters_table.csv')

In [ ]:
clusters

In [ ]:


def user_ranking(location, price, size, listing_id, clusters):
    #tmp= clusters[(clusters['listing_id'] == listing_id )]
    #rating=tmp['review_scores_rating']
    
    price_c=price_cat(price,percentiles[location])
    
    
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    cluster['ranking'] = cluster['review_scores_rating'].rank(method='min',ascending=False)
    cluster_average = cluster['review_scores_rating'].mean()
    user_rank =int(cluster[cluster['listing_id']== listing_id].iloc[0].loc['ranking'])
    return user_rank, cluster_average, cluster.shape[0]
    

user_rank, mean_rating, cluster_size=user_ranking('Hackney', 120, 'large', 102391, clusters)

print(f"Out of {cluster_size} similar properties with an average rating of {round(mean_rating/20,2)}, your listing \
is ranked n. {user_rank}")

In [ ]:
user_rank, mean_rating, cluster_size

In [ ]:
def top_rated(location, price, size, clusters, top=10):
    
    price_c=price_cat(price,percentiles[location])
    
    
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    cluster['ranking'] = cluster['review_scores_rating'].rank(method='min',ascending=False)
    
    sorted_cluster = cluster.sort_values(by='ranking')
    
    nrows= round(cluster.shape[0]/ top)
    return sorted_cluster.iloc[:nrows].listing_id
    #return sorted_cluster.iloc[:nrows].index.values.tolist()

In [ ]:
top = top_rated('Hackney', 120, 'large', clusters)

In [ ]:
def cluster_selection(location, price, size, clusters):
    price_c=price_cat(price,percentiles[location])
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    return cluster
cluster_selection('Hackney', 120, 'large', clusters)

In [ ]:
def cluster_mapping(location, price, size, clusters, listings):
    
    price_c=price_cat(price,percentiles[location])
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    
    indices = cluster.index.values.tolist()
    coordinates = listings.iloc[indices][['latitude','longitude']]
    return coordinates

In [ ]:
cluster_mapping('Hackney', 120, 'large', clusters,listings)

In [ ]:
matching_table  = clusters.drop(columns=['listing_id']).copy()

In [ ]:
matching_table

In [ ]:
matching_table = matching_table.groupby(['location','price-boroughwise','property_type']).count().reset_index().drop(columns='review_scores_rating')
matching_table['cluster_id']=matching_table.index

In [ ]:
matching_table

In [ ]:
matching_table.to_csv('clusters_table.csv')

In [6]:
percentiles = {'Lambeth': [33.0, 40.0, 50.0, 60.0, 70.0, 85.0, 100.0, 125.0, 165.0],
             'Islington': [40.0, 50.0, 65.0, 80.0, 90.0, 100.0, 120.0, 140.0, 180.0],
             'Kensington and Chelsea': [61.0, 85.0, 100.0, 119.0, 130.5, 150.0, 180.0, 220.0, 300.0],
             'Westminster': [55.0, 75.0, 95.0, 110.0, 129.0, 150.0, 180.0, 229.0, 320.0],
             'Wandsworth': [35.0, 44.0, 52.0, 68.0, 80.0, 95.0, 110.0, 140.0, 180.0],
             'Barnet': [26.0, 30.0, 37.0, 43.0, 50.0, 63.599999999999966, 76.0, 97.80000000000007, 120.0],
             'Hounslow': [27.0, 34.0, 40.0, 45.0, 55.0, 75.0, 89.0, 120.0, 160.0],
             'Camden': [45.0, 55.0, 70.0, 89.0, 100.0, 120.0, 143.0, 175.0, 230.0],
             'Richmond upon Thames': [37.0, 45.0, 55.0, 65.0, 80.0, 93.0, 110.0, 140.0, 195.0],
             'Haringey': [28.0, 32.0, 40.0, 49.0, 59.0, 75.0, 87.0, 106.60000000000014, 150.0],
             'Hackney': [34.0, 42.0, 54.0, 68.0, 80.0, 91.0, 101.0, 120.0, 160.0],
             'Southwark': [35.0, 43.0, 51.0, 65.0, 80.0, 95.0, 110.0, 135.0, 180.0],
             'Hammersmith and Fulham': [38.0, 50.0, 65.0, 79.0, 90.0, 100.0, 120.0, 150.0, 200.0],
             'Brent': [27.0, 33.0, 39.0, 47.0, 59.0, 69.59999999999991, 87.19999999999982, 110.0, 150.0],
             'Tower Hamlets': [30.0, 39.0, 46.0, 59.0, 75.0, 89.0, 100.0, 123.0, 155.0],
             'Newham': [25.0, 30.0, 35.0, 45.0, 55.0, 70.0, 90.0, 118.0, 155.0],
             'Merton': [27.0, 33.0, 40.0, 48.0, 59.0, 70.0, 94.99999999999977, 120.0, 160.0],
             'Bromley': [24.0, 28.0, 30.0, 37.20000000000002, 45.0, 51.0, 69.0, 80.0, 105.09999999999997],
             'Croydon': [22.0, 25.0, 30.0, 35.0, 42.0, 50.0, 67.0, 80.0, 104.30000000000007],
             'Lewisham': [25.0, 30.0, 35.0, 40.0, 50.0, 60.0, 75.0, 90.0, 120.0],
             'Greenwich': [25.0, 31.80000000000001, 40.0, 50.0, 65.0, 80.0, 99.0, 120.0, 160.0],
             'Ealing': [28.0, 33.0, 39.0, 45.0, 50.0, 60.0, 75.0, 100.0, 150.0],
             'Enfield': [22.0, 26.799999999999997, 30.0, 35.0, 45.0, 57.0, 70.0, 90.0, 120.0],
             'City of London': [84.70000000000002, 99.0, 119.7, 130.0, 139.5, 150.0, 169.0, 199.0, 238.2],
             'Barking and Dagenham': [20.0, 25.0, 30.0, 35.0, 40.0, 50.0, 66.6, 80.0, 120.0],
             'Waltham Forest': [25.0, 28.0, 33.0, 41.200000000000045, 50.0, 60.0, 75.0, 95.0, 120.0],
             'Hillingdon': [25.0, 28.0, 30.0, 35.0, 45.0, 50.0, 65.0, 85.0, 120.0],
             'Harrow': [23.0, 27.0, 30.0, 35.0, 40.0, 55.0, 72.99999999999997, 89.0, 120.0],
             'Redbridge': [22.0, 25.0, 27.599999999999994, 31.0, 40.0, 50.0, 55.0, 70.0, 100.0],
             'Kingston upon Thames': [30.0, 35.0, 40.0, 45.0, 52.0, 60.0, 81.0, 99.0,137.0],
             'Havering': [21.0, 25.0, 29.0, 30.800000000000004, 36.0, 45.0, 57.699999999999974, 80.0, 119.3],
             'Bexley': [20.0, 25.0, 27.0, 30.0, 34.0, 40.0, 50.0, 70.0, 88.99999999999999],
             'Sutton': [24.9, 28.0, 31.0, 35.0, 44.5, 50.0, 56.19999999999999, 65.4, 85.0]
             }


In [ ]:
top

In [ ]:
clusters[clusters['listing_id']== listing_id]

In [ ]:
test = listings[listings['id'] == int(53242)].to_dict('records')

In [ ]:
test[0]['neighbourhood_cleansed']

In [ ]:
listings.head(50)

In [7]:
def property_cat(ptype, psize):
    '''Transforms property type and size into a property cluster category'''
    if ptype != 'Entire home/apt':
        return 'room'
    elif psize < 2:
        return 'small'
    return 'large'

def price_cat(x,pctl):
    '''Transforms an absolute price into a price category, given a list of percentiles'''
    if x <pctl[1]:
        return 'very_cheap'
    elif x >=pctl[1] and x <pctl[3]:
        return 'cheap'
    elif x >=pctl[3] and x <pctl[5]:
        return 'average'
    elif x >=pctl[5] and x <pctl[7]:
        return 'expensive'
    return 'very_expensive'

def user_ranking_new(location, price, ptype, psize, listing_id, clusters, percentiles):
    '''Takes as input a neighborhood, a price, a listing property type, the listing id and the clusters dataframe
    returns:
    - the listing's ranking within its cluster
    - the cluster's average rating
    - the cluster size
    '''

    price_c=price_cat(price,percentiles[location])
    size_c = property_cat(ptype, psize)

    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size_c )\
            ].reset_index()
    cluster['ranking'] = cluster['review_scores_rating'].rank(method='min',ascending=False,pct=True)
    cluster_average = cluster['review_scores_rating'].mean()
    user_rank =float(cluster[cluster['listing_id']== listing_id]['ranking'])
    return user_rank, cluster_average, cluster['review_scores_rating']

aa, bb, cc =user_ranking_new('Lambeth', 55, 'Private room', 1 ,53242, clusters, percentiles)

In [70]:
#clusters=pd.read_csv('../data/jan/clusters.csv')

price_c=price_cat(55,percentiles['Lambeth'])
size_c = property_cat('Private room', 1)

cluster = clusters[(clusters['location'] == 'Lambeth' ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size_c )\
            ].reset_index()
cluster['ranking'] = cluster['review_scores_rating'].rank(method='min',ascending=False,pct=True)

In [4]:
clusters=pd.read_csv('../data/jan/clusters.csv')

In [77]:
float(cluster[cluster['listing_id']==53242]['ranking'])

KeyError: (0, 'ranking')

In [80]:
clusters

,Unnamed: 0,location,price-boroughwise,property_type,review_scores_rating,listing_id,lat,lon,cluster
0,0,Lambeth,expensive,small,92.0,11551.0,51.46225,-0.11732,L:Lambeth_P:expensive_S:small
1,33,Lambeth,cheap,room,98.0,53242.0,51.47075,-0.12913,L:Lambeth_P:cheap_S:room
2,34,Lambeth,expensive,small,96.0,53245.0,51.47246,-0.12727,L:Lambeth_P:expensive_S:small
3,50,Lambeth,cheap,room,96.0,72913.0,51.48064,-0.11439,L:Lambeth_P:cheap_S:room
4,51,Lambeth,cheap,large,98.0,73431.0,51.46295,-0.11200,L:Lambeth_P:cheap_S:large
...,...,...,...,...,...,...,...,...,...
51743,51047,Sutton,expensive,small,100.0,39963193.0,51.35770,-0.13601,L:Sutton_P:expensive_S:small
51744,51074,Sutton,expensive,small,98.0,39988769.0,51.35737,-0.21677,L:Sutton_P:expensive_S:small
51745,51098,Sutton,very_expensive,small,40.0,40019636.0,51.37853,-0.21887,L:Sutton_P:very_expensive_S:small
51746,51255,Sutton,expensive,large,60.0,40147104.0,51.34812,-0.19767,L:Sutton_P:expensive_S:large


In [66]:
user_rank =float(cluster[cluster['listing_id']== 53242].reset_index()['ranking'])

In [67]:
user_rank

0.2579034941763727

In [37]:
size_c

'room'

In [19]:
cluster_average = cluster['review_scores_rating'].mean()

In [20]:
cluster_average

93.85690515806988

In [42]:
cluster[cluster['listing_id']== 72913].iloc[0].loc['ranking']

0.41430948419301167

In [18]:
cl =cc.tolist()

In [20]:
cl.append(95.6)

In [21]:
cl

[98.0,
 96.0,
 97.0,
 97.0,
 92.0,
 93.0,
 95.0,
 87.0,
 93.0,
 98.0,
 90.0,
 100.0,
 93.0,
 96.0,
 98.0,
 97.0,
 100.0,
 99.0,
 96.0,
 89.0,
 89.0,
 90.0,
 91.0,
 93.0,
 96.0,
 95.0,
 96.0,
 91.0,
 99.0,
 96.0,
 98.0,
 82.0,
 95.0,
 99.0,
 94.0,
 99.0,
 96.0,
 94.0,
 93.0,
 98.0,
 87.0,
 96.0,
 97.0,
 98.0,
 92.0,
 96.0,
 96.0,
 95.0,
 92.0,
 92.0,
 92.0,
 97.0,
 98.0,
 90.0,
 96.0,
 97.0,
 94.0,
 89.0,
 92.0,
 96.0,
 98.0,
 93.0,
 95.0,
 99.0,
 95.0,
 75.0,
 96.0,
 100.0,
 94.0,
 92.0,
 90.0,
 97.0,
 98.0,
 100.0,
 93.0,
 91.0,
 99.0,
 91.0,
 93.0,
 97.0,
 93.0,
 100.0,
 99.0,
 94.0,
 87.0,
 88.0,
 97.0,
 96.0,
 84.0,
 85.0,
 100.0,
 98.0,
 90.0,
 100.0,
 78.0,
 92.0,
 89.0,
 95.0,
 94.0,
 94.0,
 94.0,
 87.0,
 98.0,
 99.0,
 99.0,
 87.0,
 96.0,
 95.0,
 98.0,
 80.0,
 82.0,
 99.0,
 86.0,
 93.0,
 87.0,
 87.0,
 98.0,
 93.0,
 88.0,
 100.0,
 97.0,
 94.0,
 93.0,
 83.0,
 92.0,
 80.0,
 100.0,
 100.0,
 98.0,
 91.0,
 98.0,
 97.0,
 93.0,
 80.0,
 81.0,
 81.0,
 90.0,
 98.0,
 97.0,
 94.0,
 89.0,
 98

In [24]:
pd.Series(cl).rank(method='min',ascending=False,pct=True).iloc[-1]

0.49335548172757476

In [25]:
cc

0       98.0
1       96.0
2       97.0
3       97.0
4       92.0
5       93.0
6       95.0
7       87.0
8       93.0
9       98.0
10      90.0
11     100.0
12      93.0
13      96.0
14      98.0
15      97.0
16     100.0
17      99.0
18      96.0
19      89.0
20      89.0
21      90.0
22      91.0
23      93.0
24      96.0
25      95.0
26      96.0
27      91.0
28      99.0
29      96.0
30      98.0
31      82.0
32      95.0
33      99.0
34      94.0
35      99.0
36      96.0
37      94.0
38      93.0
39      98.0
40      87.0
41      96.0
42      97.0
43      98.0
44      92.0
45      96.0
46      96.0
47      95.0
48      92.0
49      92.0
50      92.0
51      97.0
52      98.0
53      90.0
54      96.0
55      97.0
56      94.0
57      89.0
58      92.0
59      96.0
60      98.0
61      93.0
62      95.0
63      99.0
64      95.0
65      75.0
66      96.0
67     100.0
68      94.0
69      92.0
70      90.0
71      97.0
72      98.0
73     100.0
74      93.0
75      91.0
76      99.0

In [26]:
def get_ranking(series, value):
    tmp = series.tolist()
    tmp.append(value)
    ranking = pd.Series(tmp).rank(method='min',ascending=False,pct=True).iloc[-1]
    return ranking

In [30]:
get_ranking(cc, 91)

0.739202657807309